In [ ]:
!pip install openpyxl

import pandas as pd
import numpy as np


In [ ]:
df = pd.read_excel('/content/Online Retail.xlsx', engine='openpyxl')

df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df = df.dropna(subset=['CustomerID'])
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')].copy()  # <-- Explicit copy

df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

In [ ]:
clv_data = df.groupby('CustomerID').agg({
    'InvoiceNo': 'nunique',
    'Quantity': 'sum',
    'TotalPrice': 'sum',
    'InvoiceDate': ['min', 'max']
})

clv_data.columns = ['TotalPurchases', 'TotalUnits', 'TotalRevenue', 'FirstPurchase', 'LastPurchase']


In [ ]:
clv_data['CustomerAge'] = (clv_data['LastPurchase'] - clv_data['FirstPurchase']).dt.days + 1
clv_data['RevenuePerDay'] = clv_data['TotalRevenue'] / clv_data['CustomerAge']
clv_data['CLV'] = clv_data['RevenuePerDay'] * 365


In [ ]:
clv_data = clv_data.reset_index()
clv_data.to_csv('customer_lifetime_value_report.csv', index=False)


In [ ]:
clv_data

,CustomerID,TotalPurchases,TotalUnits,TotalRevenue,FirstPurchase,LastPurchase,CustomerAge,RevenuePerDay,CLV
0,12346.0,1,74215,77183.60,2011-01-18 10:01:00,2011-01-18 10:01:00,1,77183.600000,2.817201e+07
1,12347.0,7,2458,4310.00,2010-12-07 14:57:00,2011-12-07 15:52:00,366,11.775956,4.298224e+03
2,12348.0,4,2341,1797.24,2010-12-16 19:09:00,2011-09-25 13:13:00,283,6.350671,2.317995e+03
3,12349.0,1,631,1757.55,2011-11-21 09:51:00,2011-11-21 09:51:00,1,1757.550000,6.415058e+05
4,12350.0,1,197,334.40,2011-02-02 16:01:00,2011-02-02 16:01:00,1,334.400000,1.220560e+05
...,...,...,...,...,...,...,...,...,...
4334,18280.0,1,45,180.60,2011-03-07 09:52:00,2011-03-07 09:52:00,1,180.600000,6.591900e+04
4335,18281.0,1,54,80.82,2011-06-12 10:53:00,2011-06-12 10:53:00,1,80.820000,2.949930e+04
4336,18282.0,2,103,178.05,2011-08-05 13:35:00,2011-12-02 11:43:00,119,1.496218,5.461197e+02
4337,18283.0,16,1397,2094.88,2011-01-06 14:14:00,2011-12-06 12:02:00,334,6.272096,2.289315e+03
